In [1]:
import pandas as pd
data = pd.read_csv("Disaster.csv")
data.columns

Index(['Name', 'UserName', 'Timestamp', 'Verified', 'Tweets', 'Comments',
       'Retweets', 'Likes', 'Impressions', 'Tags', 'Tweet Link', 'Tweet ID',
       'Disaster'],
      dtype='object')

In [2]:
texts = data['Tweets'].tolist()
labels = data['Disaster'].tolist()

In [3]:
data.Disaster.value_counts()

Disaster
Drought       770
Wildfire      540
Earthquake    500
Floods        436
Hurricanes    178
Tornadoes     135
Name: count, dtype: int64

In [4]:
# Create a mapping dictionary for disaster types
disaster_mapping = {
    'Drought': 0,
    'Wildfire': 1,
    'Earthquake': 2,
    'Floods': 3,
    'Hurricanes': 4,
    'Tornadoes': 5
}

# Apply the mapping to the Disaster column
data['Disaster'] = data['Disaster'].map(disaster_mapping)

In [5]:
data.Disaster.value_counts()

Disaster
0    770
1    540
2    500
3    436
4    178
5    135
Name: count, dtype: int64

In [6]:
import torch
from sklearn.model_selection import train_test_split
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

# Load the model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

c:\Users\LENOVO\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Plea

In [7]:
# Data Preparation: Select 1000 random samples from the dataset
data = data.sample(1000, random_state=42 )

# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['Tweets'], data['Disaster'], test_size=0.2, random_state=42)

In [8]:
# Tokenize the texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)

In [9]:
# Convert to torch tensors
class DisasterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
train_dataset = DisasterDataset(train_encodings, train_labels.tolist())
test_dataset = DisasterDataset(test_encodings, test_labels.tolist())

# Create a DataLoader
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

In [11]:
# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(3):  # Training for 3 epochs
    for batch in train_dataloader:
        optimizer.zero_grad()
        
        inputs = {key: val for key, val in batch.items() if key != 'labels'}
        labels = batch['labels']
        
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

c:\Users\LENOVO\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0, Loss: 1.8611637353897095
Epoch: 0, Loss: 1.6825507879257202
Epoch: 0, Loss: 1.6659104824066162
Epoch: 0, Loss: 1.7324029207229614
Epoch: 0, Loss: 1.352903962135315
Epoch: 0, Loss: 1.8923814296722412
Epoch: 0, Loss: 1.5810589790344238
Epoch: 0, Loss: 1.4255423545837402
Epoch: 0, Loss: 1.8504620790481567
Epoch: 0, Loss: 1.4758139848709106
Epoch: 0, Loss: 1.4438576698303223
Epoch: 0, Loss: 1.1957381963729858
Epoch: 0, Loss: 1.3832604885101318
Epoch: 0, Loss: 1.1025046110153198
Epoch: 0, Loss: 1.2231907844543457
Epoch: 0, Loss: 1.3225617408752441
Epoch: 0, Loss: 1.1537021398544312
Epoch: 0, Loss: 1.175886631011963
Epoch: 0, Loss: 1.3046786785125732
Epoch: 0, Loss: 1.0497820377349854
Epoch: 0, Loss: 0.7837339639663696
Epoch: 0, Loss: 0.7796065211296082
Epoch: 0, Loss: 0.765639066696167
Epoch: 0, Loss: 0.7415425181388855
Epoch: 0, Loss: 0.7704342007637024
Epoch: 0, Loss: 0.8213167190551758
Epoch: 0, Loss: 0.7990801334381104
Epoch: 0, Loss: 0.694758951663971
Epoch: 0, Loss: 0.494794